In [1]:
import os
import pandas as pd
import scanpy as sc 
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc 

In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(dpi=100)

In [3]:
os.chdir("/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_h5")

In [1]:
import glob
H5data=[]
for line in  glob.glob("*h5"):
    # print(line)
    H5data.append(line)
print(H5data)

### Doublet rate calculation

In [5]:
import scrublet as scr
import os
import pandas as pd
import scanpy as sc  
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import rcParams
# import scanpy as sc 
%matplotlib inline
# import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

plt.rc('font', size=14)
plt.rcParams['pdf.fonttype'] = 42

In [2]:
filenames =H5data
adatadict_sc={}
# adatas = []
scrub_doublet_rate=pd.DataFrame()
i=0
for filename in filenames: 
    
    dat1=sc.read_10x_h5(filename)
    name=filename.rsplit("_")[0]
    dat1.obs_names=name+"_"+dat1.obs_names 
    dat1.obs["Name"]=name 
    dat1.var_names_make_unique() 
    mat_test1=pd.DataFrame(data=dat1.X.todense(),index=dat1.obs_names,columns=dat1.var_names)
    scrub1 = scr.Scrublet(mat_test1)
    doublet_scores, predicted_doublets = scrub1.scrub_doublets(min_counts=2, min_cells=3, min_gene_variability_pctl=85, n_prin_comps=30)
    
    dat1.uns['doublet_scores'] =doublet_scores
    dat1.uns['predicted_doublets'] = predicted_doublets 
    print (name,scrub1.detected_doublet_rate_)
   
    scrub_doublet_rate.loc[i,"name"]=name
    scrub_doublet_rate.loc[i,"rate"]=scrub1.detected_doublet_rate_
    i+=1

    adatadict_sc[name]=dat1
    

In [3]:
scrub_doublet_rate

In [5]:
# adatadict_sc.values()

In [10]:
adatadict_sc2=adatadict_sc.copy()

In [11]:

from anndata import concat

adata_list = list(adatadict_sc2.values())

adata_merged = concat(adata_list, join='outer')


In [6]:
# adatadict_sc

In [13]:
# adata_merged=[]

In [7]:
adata_list = []
# Transfer the 'uns' contents to 'obs' and collect the modified objects in a list
for name, adata in adatadict_sc2.items():
    print(name)
    for key in adata.uns.keys():
        print(key)
        adata.obs['doublet_scores'] = adata.uns['doublet_scores']
        adata.obs['predicted_doublets'] = adata.uns['predicted_doublets']
        print(adata)
    adata_list.append(adata)
# Concatenate the Anndata objects
adata_m = concat(adata_list, join='outer')

In [15]:
adata_m

AnnData object with n_obs × n_vars = 267657 × 36601
    obs: 'Name', 'doublet_scores', 'predicted_doublets'

In [16]:
# adata_merged2=pd.DataFrame(data=adata_merged.X.todense(),index=adata_merged.obs_names,columns=adata_merged.var_names)
# scrub_merge= scr.Scrublet(adata_merged2)
# doublet_scores2, predicted_doublets2 = scrub_merge.scrub_doublets(min_counts=2, min_cells=3, min_gene_variability_pctl=85, n_prin_comps=30)
# adata_merged.uns['doublet_scores'] =doublet_scores2
# adata_merged.uns['predicted_doublets'] = predicted_doublets2 

In [17]:
# adata_list = []

# # Transfer the 'uns' contents to 'obs' and collect the modified objects in a list
# for name, adata in adatadict_sc.items():
#     for key in adata.uns.keys():
#         adata.obs[key] = adata.uns[key]
#     adata_list.append(adata)

In [18]:
adata_m.obs['predicted_doublets'].value_counts()

False    257571
True      10086
Name: predicted_doublets, dtype: int64

### Celltypist

In [19]:
import scanpy as sc
import celltypist
from celltypist import models

In [20]:
sc.pp.normalize_total(adata_m, target_sum=1e4)
sc.pp.log1p(adata_m)

normalizing counts per cell
    finished (0:00:02)


In [21]:
model = models.Model.load(model = 'Immune_All_Low.pkl')

In [22]:
predictions = celltypist.annotate(adata_m, model = 'Immune_All_Low.pkl', majority_voting = True)

🔬 Input data has 267657 cells and 36601 genes
🔗 Matching reference genes in the model
🧬 6147 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, construct one before the over-clustering


filtered out 6078 genes that are detected in less than 5 cells
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:06)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:01:09)
computing neighbors
    using 'X_pca' with n_pcs = 50


/data1/maolp/mamba/envs/Pyscan/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/data1/maolp/mamba/envs/Pyscan/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/data1/maolp/mamba/envs/Pyscan/lib/python3.9/site-packages/umap/distances.py:108

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:46)


⛓️ Over-clustering input data with resolution set to 30


running Leiden clustering
    finished: found 537 clusters and added
    'over_clustering', the cluster labels (adata.obs, categorical) (0:05:18)


🗳️ Majority voting the predictions
✅ Majority voting done!


In [23]:
predictions.predicted_labels.to_csv("../All_scanpyData/All_Immu_Prediction.csv") 

In [24]:
adata = predictions.to_adata()

In [25]:
adata

AnnData object with n_obs × n_vars = 267657 × 36601
    obs: 'Name', 'doublet_scores', 'predicted_doublets', 'predicted_labels', 'over_clustering', 'majority_voting', 'conf_score'
    uns: 'log1p', 'neighbors', 'leiden'
    obsm: 'X_pca'
    obsp: 'connectivities', 'distances'

In [26]:
adata.obs

,Name,doublet_scores,predicted_doublets,predicted_labels,over_clustering,majority_voting,conf_score
Z48_AAACCTGAGATAGGAG-1,Z48,0.065421,False,Naive B cells,107,Naive B cells,0.868831
Z48_AAACCTGAGGCACATG-1,Z48,0.150350,False,Late erythroid,254,Erythrocytes,0.669659
Z48_AAACCTGAGGTGCAAC-1,Z48,0.034215,False,Erythrocytes,136,Erythrocytes,0.656841
Z48_AAACCTGCAAGAGTCG-1,Z48,0.074860,False,Erythrocytes,278,Erythrocytes,0.393525
Z48_AAACCTGCACACCGAC-1,Z48,0.215596,False,Late erythroid,167,Erythrocytes,0.780110
...,...,...,...,...,...,...,...
Z123_TTTGTCATCTAACTCT-1,Z123,0.132340,False,Erythrocytes,76,Late erythroid,0.405928
Z123_TTTGTCATCTACTTAC-1,Z123,0.047110,False,Tcm/Naive helper T cells,313,Tcm/Naive helper T cells,0.879585
Z123_TTTGTCATCTCCCTGA-1,Z123,0.018881,False,Naive B cells,144,Naive B cells,0.654993
Z123_TTTGTCATCTGGTATG-1,Z123,0.473171,True,Classical monocytes,45,Classical monocytes,0.721308


### Save First Data

In [27]:
!mkdir -p "/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData"

In [28]:
adata_m.write("/home/maolp/data5/Gaofeng_All_matrix/Allcount/All_scanpyData/D1_scanpy_merge_doubulet.h5ad")

... storing 'Name' as categorical


In [29]:
adata.write("/home/maolp/data5/Gaofeng_All_matrix/Allcount//All_scanpyData/D1_scanpy_merge_doubulet_celltypist.h5ad")